# Prepare Environment

In [ ]:
from os import path
from utils.notebook_helpers import select_device, show_in_netron
from models.cnv import CNV

data_dir = path.join('..', 'data')
model_name = 'CNV'
model_dir = 'exports'
device = select_device()

num_classes = 3
weight_bit_width = 1
act_bit_width = 1
in_bit_width = 8
in_ch = 3
seed = 0

print('Device:', device)


# Load Dataset
When used with the Gesture Dataset, make sure you have the correct folder structure as stated in CustomDataSet Docstring:

>    The file structure shall look like this:
>    
>    /path to gesten-datensatz repository/gesten-datensatz/gesture/source/train/start/0.jpg
>
>    /path to gesten-datensatz repository/gesten-datensatz/gesture/source/train/start/1.jpg
>
>    ...
>
>    /path to gesten-datensatz repository/gesten-datensatz/gesture/source/train/stop/0.jpg
>
>    ...
>
>    /path to gesten-datensatz repository/gesten-datensatz/gesture/source/test/start/0.jpg
>
>    ...
>
>    /path to gesten-datensatz repository/gesten-datensatz/gesture/source/test/stop/0.jpg

In [ ]:
from datasets.custom import CustomDataSet

dataset = CustomDataSet(root_dir=data_dir, dataset='custom_dataset', image_size=64, batch_size=32, num_workers=0)


# Show Class Ids

In [ ]:
dataset.classes


# Training

In [ ]:
from trainers.basic import Trainer

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch, seed)
model_path = path.join(model_dir, f'{model_name}.pth')
Trainer(model=model, save_path=model_path, train_dataloader=dataset.train_loader, device=device, epochs=10, print_every=10).train()


## Measure accuracy

In [ ]:
from validators.basic import Validator

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch, seed)
Validator(model=model, load_path=model_path, test_dataloader=dataset.test_loader, device=device, print_test_information=True).validate()


## FINN Export

In [ ]:
from exporters.onnx import ONNXExporter

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch, seed)
onnx_path = path.join(model_dir, f'{model_name}.onnx')
ONNXExporter(model=model, load_path=model_path, save_path=onnx_path).export()

show_in_netron(onnx_path)
